In [ ]:
### THE CODE BELOW WORKS ONLY IN THE GCP GOOGLE FUNCTIONS; MUST COPY, PASTE AND RUN THERE
### This code only updates the data already in the SQL Database (i.e. does not serve as basic information and does not create new tables)

# requirements:
# functions_framework
# SQLAlchemy
# PyMySQL
# pandas
# requests

import functions_framework
import pandas as pd
import sqlalchemy
import requests

@functions_framework.http
def insert_weather_parameters(request):
  connection_string = connection()
  insert_weather_data_in_SQL(connection_string)
  return "Data successfully added - all Ok"

def connection():
    schema = "gans_data_pipeline"  # update "gans_data_pipeline" to your actual MySql database (schema) name
    host = "127.0.0.1"  # update with your own cloud SQL instance public IP address
    user = "root"  # update with your MySql username
    password = "password" # update "password" with your actual MySql password
    port = 3306  # update with your own default MySQL port, if different
    connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
    return connection_string

def insert_weather_data_in_SQL(connection_string):
    city_name = []
    forecast_time =[]
    temp_c=[]
    temp_feels =[]
    temp_min =[]
    temp_max = []
    weather_desc =[]
    cloudiness =[]
    humidity_p =[]
    prob_rain =[]
    wind_speed =[]
    cities_from_sql = pd.read_sql("cities", con=connection_string)
    cities = cities_from_sql["city"].to_list()
    
    #in the "openweather_response line 29, replace [API_KEY_HERE] with your actual API key from OpenWeatherMap
    for city in cities:
        openweather_response = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid=[API_KEY_HERE]&units=metric")
        openweather_response_json = openweather_response.json()
        
        for i in openweather_response_json['list']:
          forecast_time.append(i.get('dt_txt', None)),
          temp_c.append(i['main'].get('temp', None)),
          temp_feels.append(i['main'].get('feels_like', None)),
          temp_min.append(i['main'].get('temp_min', None)),
          temp_max.append(i['main'].get('temp_max', None)),
          weather_desc.append(i['weather'][0].get('main', None)),
          cloudiness.append(i['clouds'].get('all', None)),
          humidity_p.append(i['main'].get('humidity', None)),
          prob_rain.append(int(i.get('pop', None))*100),
          wind_speed.append(i['wind'].get('speed', None)),
          city_name.append(city)

    city_weather_df = (pd.DataFrame(list(zip(city_name, forecast_time,
                                            temp_c, temp_feels, temp_min,
                                            temp_max, weather_desc, cloudiness,
                                            humidity_p, prob_rain, wind_speed)),
                                  columns=["city", "forecast_time",
                                            "temp_in_c", "temp_feel", "temp_min",
                                            "temp_max", "weather_description", "cloudiness_percent",
                                            "humidity_percent", "rain_prob_percent", "wind_speed_m_sec"]))
    
    merged_weather_df = city_weather_df.merge(cities_from_sql,
                                   on = "city",
                                   how="left")
    
    weather_df = merged_weather_df.drop(columns=["city","country_id", "latitude","longitude"])
    
    weather_df['forecast_time']=pd.to_datetime(weather_df['forecast_time'])

    weather_df.to_sql('weather',
                  if_exists='append',
                  con=connection_string,
                  index=False)